<a href="https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/create_graphic_mlo_one_month_keelingcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create a graphic appearing on the Keeling Curve website
### **Graphic: CO<sub>2</sub> record at Mauna Loa, Hawaii for the last month**

#### This notebook creates customizable PDF and PNG images of a graphic appearing on the front page of [keelingcurve.ucsd.edu](keelingcurve.ucsd.edu).

For an overview of the Keeling Curve notebooks, data and graphics licenses, and how to use the notebooks, see [notebooks overview](https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/overview_of_notebooks_keelingcurve.ipynb).

**To use the notebook**, first run all the cells. In Google Colab, select from the top menu and select Runtime -> Run all. Then the buttons will appear to click on which creates and downloads the graphics.

Direct questions to [webmaster-co2o2-sio@ucsd.edu](mailto:webmaster-co2o2-sio@ucsd.edu)

---

### [Go to creating the graphic section](#button-to-create-and-save-the-graphic)

---

### Sample of the graphic produced by this notebook

![title](https://github.com/sio-co2o2/keelingcurve_notebooks/blob/main/images/sample_graphics/mlo_one_month.png?raw=1)

## **Notebook Code**

### Import python packages

In [32]:
import functools
import re
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import pathlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import ticker
from matplotlib import rc,rcParams
from matplotlib.path import Path
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon
import matplotlib.dates as mdates
import matplotlib.lines as mlines
from scipy.interpolate import UnivariateSpline
from scipy import interpolate
import matplotlib.transforms
from matplotlib import animation
from ipywidgets import widgets
from IPython.display import HTML, display
from base64 import b64encode
from PIL import Image
import urllib
import os

%matplotlib inline

In [33]:
# This package is used to convert a vector svg into a png

try:
  from cairosvg import svg2png
except:
  ! pip install cairosvg
  from cairosvg import svg2png

In [34]:
# This import is to enable Google Colab to save files ane then download them

# This import does not exist unless the notebook is run in Google Colab
# Put in a try except block if user wants to use notebook off of Google Colab

try:
    from google.colab import files
except:
    pass

### Set directories and file names

In [35]:
# Read in data from github repository
data_file = 'https://raw.githubusercontent.com/sio-co2o2/keelingcurve_notebooks/main/preliminary_data/mlo/mlo_one_month_span.csv'

logo_file = 'https://github.com/sio-co2o2/keelingcurve_notebooks/raw/main/images/logos/ucsd_sio_logo.svg'

plot_dir = pathlib.Path('./plots')
plot_dir.mkdir(exist_ok=True)

plot_basename = 'mlo_one_month'

pdf_file = plot_dir / f"{plot_basename}.pdf"
png_file = plot_dir / f"{plot_basename}.png"

## **Load in data and prepare it for plotting**

### Load in MLO record

In [36]:
df = pd.read_csv(data_file,sep=',',comment='"')
df.head()

,date_hr,co2_hr,date_dy,co2_dy,date_wk,co2_wk
0,2023.074037,419.40,2023.075342,419.71,2023.073973,419.66
1,2023.074115,419.33,2023.078082,420.03,2023.093151,419.68
2,2023.074288,419.41,2023.080822,419.34,2023.112329,420.63
3,2023.074380,419.25,2023.083562,419.23,2023.131507,420.19
4,2023.074496,419.36,2023.086301,419.31,2023.150685,420.76


### Break into dataframes for each time frequency

In [37]:
df_hourly = df[['date_hr', 'co2_hr']].copy()
df_daily = df[['date_dy', 'co2_dy']].copy()
df_wk = df[['date_wk', 'co2_wk']].copy()

### Remove rows with fill values NaN

In [38]:
df_hourly = df_hourly[df_hourly['co2_hr'].notnull()]
df_daily = df_daily[df_daily['co2_dy'].notnull()]
df_weekly = df_wk[df_wk['co2_wk'].notnull()]

## Extract out Maunakea (MKO) data 

Need to differentiate MKO data when plotting

### Get start and end decimal dates of MKO data (12/1/2022 to 3/8/2023)

In [39]:
mauna_kea_start_date = datetime(2022, 12, 1)

year = mauna_kea_start_date.year
boy = datetime(year, 1, 1)
eoy = datetime(year + 1, 1, 1)
mko_decimal_start_date = year + ((mauna_kea_start_date - boy).total_seconds() / ((eoy - boy).total_seconds()))

mauna_kea_end_date = datetime(2023, 3, 8)

year = mauna_kea_end_date.year
boy = datetime(year, 1, 1)
eoy = datetime(year + 1, 1, 1)
mko_decimal_end_date = year + ((mauna_kea_end_date - boy).total_seconds() / ((eoy - boy).total_seconds()))


### Filter data to get MKO data

In [40]:
df_hourly_mko = df_hourly[(df_hourly['date_hr'] >= mko_decimal_start_date) & (df_hourly['date_hr'] <= mko_decimal_end_date)]

df_daily_mko = df_daily[(df_daily['date_dy'] >= mko_decimal_start_date) & (df_daily['date_dy'] <= mko_decimal_end_date)]

df_weekly_mko = df_weekly[(df_weekly['date_wk'] >= mko_decimal_start_date) & (df_weekly['date_wk'] <= mko_decimal_end_date)]


### Convert to numpy arrays for plotting

In [41]:
date_hourly = df_hourly['date_hr'].to_numpy()
co2_hourly = df_hourly['co2_hr'].to_numpy()

date_daily = df_daily['date_dy'].to_numpy()
co2_daily = df_daily['co2_dy'].to_numpy()

date_weekly = df_weekly['date_wk'].to_numpy()
co2_weekly = df_weekly['co2_wk'].to_numpy()

date_hourly_mko = df_hourly_mko['date_hr'].to_numpy()
co2_hourly_mko = df_hourly_mko['co2_hr'].to_numpy()

date_daily_mko = df_daily_mko['date_dy'].to_numpy()
co2_daily_mko = df_daily_mko['co2_dy'].to_numpy()

date_weekly_mko = df_weekly_mko['date_wk'].to_numpy()
co2_weekly_mko = df_weekly_mko['co2_wk'].to_numpy()


## **Define accessory functions**

## Functions to convert dates into various formats
Used for plot limits and labeling

### Function to convert a datetime to a float

In [42]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def dt2t(adatetime):
    """
    Convert adatetime into a float. The integer part of the float should
    represent the year.
    Order should be preserved. If adate<bdate, then d2t(adate)<d2t(bdate)
    time distances should be preserved: If bdate-adate=ddate-cdate then
    dt2t(bdate)-dt2t(adate) = dt2t(ddate)-dt2t(cdate)
    """
    year = adatetime.year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    
    return year + ((adatetime - boy).total_seconds() / ((eoy - boy).total_seconds()))

### Function to convert a float to a datetime

In [43]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def t2dt(atime):
    """
    Convert atime (a float) to DT.datetime
    This is the inverse of dt2t.
    assert dt2t(t2dt(atime)) == atime
    """
    year = int(atime)
    remainder = atime - year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    seconds = remainder * (eoy - boy).total_seconds()
    
    return boy + timedelta(seconds=seconds)

### Function to create various date versions for today's date

In [44]:
def get_todays_date_variations():
    
    now = datetime.now()
    todays_decimal =  dt2t(now)
    
    today = date.today()

    todays_day = today.day
    todays_month = today.strftime("%B")
    todays_year = today.year
    todays_date_moyr = today.strftime("%B %Y")
    todays_date_modyyr = f"{todays_month} {todays_day}, {todays_year}"

    return todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year

### Function to get the end date of data

In [45]:
def get_data_end_date(date_data):
    last_date = np.max(date_data)
    last_date_datetime = t2dt(last_date)
    
    # Convert date format to month_name day, year
    day = last_date_datetime.day
    month = last_date_datetime.strftime("%B")
    year = last_date_datetime.year
    date_modyyr = f"{month} {day}, {year}"
    return date_modyyr

<a name="functions-to-modify-graphic"></a>
## **Define functions that can modify the graphic**

### Function to set global fonts and linewidth properties

In [46]:
def set_matplotlib_properties():
    
    # Set default properties for matplotlib
    
    # Reset rcparams in case modified defaults in another notebook during same session 
    plt.rcParams.update(plt.rcParamsDefault)

    plt.rcParams.update({'axes.linewidth':1.5})
    
    plt.rcParams.update({
        "text.usetex": False,
        "font.family": "sans-serif",
        "font.weight":  "normal",
        "font.sans-serif": ["Arial", "Tahoma", "Helvetica","FreeSans", "NimbusSans", "LiberationSans","DejaVu Sans"],
        "mathtext.default":'regular',
        "mathtext.fontset": "dejavusans"
    })
        
    # http://phyletica.org/matplotlib-fonts/
    # This causes matplotlib to use Type 42 (a.k.a. TrueType) fonts 
    # for PostScript and PDF files. This allows you to avoid Type 3 fonts.
    # Turning on usetex also works
    # Needed on Ubuntu for pdf fonts 
    plt.rcParams.update({
      'pdf.fonttype': 42,
        'ps.fonttype': 42 
    })

### Function to set axes properties of the graphic

In [47]:
def set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel):

    # ---------------------------------
    # Plot properties for website plots
    # ---------------------------------

    # Allow room at top for the 2 titles
    fig.subplots_adjust(top=0.85)

    ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)

    ax.tick_params(axis='x', labelsize=12, pad=5)
    ax.tick_params(axis='y', labelsize=16)

    ax.tick_params(axis='x', which='major', direction='in', length=6, width=1)
    ax.tick_params(axis='y', which='major', direction='in', length=8, width=1)

    ax.xaxis.set_major_formatter(mdates.DateFormatter('%-d'))
    
    fmt_every_day = mdates.DayLocator(interval=1)
    ax.xaxis.set_major_locator(fmt_every_day)
    
    tick_spacing = 5
    ax.yaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

    labels = ax.get_xticklabels() + ax.get_yticklabels()
    for label in labels: label.set_fontweight('bold') 

    ax.tick_params(which='minor', direction='in', length=4)
    
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    
    ax.tick_params(axis='x', which='minor', bottom=False)
    
    ax.set_xlabel(xlabel, fontweight='bold', fontsize=18, labelpad=5)
    ax.set_ylabel(ylabel, fontweight='bold', fontsize=18, labelpad=5)

    # Set axes limits last 
    # If did before, setting xtick labels past xmin & xmax would have
    # extended the plot limits
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

### Function to create text tick labels

In [48]:
def create_xtick_labels(ax, xmin, xmax):
    
    # Move tick labels so left aligned with tick mark
    # to represent the day as starting at the tick mark
    for tick in ax.xaxis.get_majorticklabels():
        tick.set_horizontalalignment("left")
        
    # Place the starting and ending months below the xaxis tick labels
    start_month = xmin.month
    start_month = date(1900, start_month, 1).strftime('%b')

    label_text = start_month
    ax.annotate(label_text, xy=(0, -0.06), xycoords='axes fraction', fontsize=10, fontweight='bold',
                horizontalalignment='left', verticalalignment='top')

    end_month = xmax.month
    end_month = date(1900, end_month, 1).strftime('%b')
    
    label_text = end_month
    ax.annotate(label_text, xy=(0.97, -0.06), xycoords='axes fraction', fontsize=10, fontweight='bold',
                horizontalalignment='left', verticalalignment='top')

    ax.tick_params(axis='x', pad=7)

### Function to place the titles

In [49]:
def add_plot_title(ax, title1, title2):

    ax.annotate(title1, xy=(0, 1.15), xycoords='axes fraction', fontsize=14,
                horizontalalignment='left', verticalalignment='top', fontweight="normal")
    ax.annotate(title2, xy=(0, 1.07), xycoords='axes fraction', fontsize=18,
                horizontalalignment='left', verticalalignment='top', fontweight = 'bold')

### Function to add an inset label

In [50]:
def add_inset_label(ax, label_start, today):
    label_text = f"{label_start} ending {today}"
    ax.annotate(label_text, xy=(0.03, 0.9), xycoords='axes fraction', fontsize=12,
                horizontalalignment='left', verticalalignment='top')

In [51]:
def add_inset_label_mko(ax, label_text):
    blue = '#1e47b0'
    ax.annotate(label_text, xy=(0.03, 0.8), xycoords='axes fraction', fontsize=12,
                horizontalalignment='left', verticalalignment='top', fontweight="bold", color=blue)

### Function to add legend

In [52]:
def add_legend_labels(ax):

    legend_1 = 'Hourly average'
    legend_2 = 'Daily average'
    legend_3 = 'Weekly average'
    
    legend_properties = {'weight':'bold'}

    black_dot = mlines.Line2D([], [], marker = 'o', color='black',markersize=4, 
             markerfacecolor='black', markeredgecolor='black', markeredgewidth=0, linestyle='None')

    black_filled_circle = mlines.Line2D([], [], marker = 'o', color='black', markersize=10, 
             markerfacecolor='black', markeredgecolor='black', markeredgewidth=0, linestyle='None')
   
    black_filled_circle_lg = mlines.Line2D([], [], marker = 'o', color='black', markersize=12, 
             markerfacecolor='none', markeredgecolor='black', markeredgewidth=2.0, linestyle='None')

    # Use upper right for legend
    plt.rc('legend',fontsize=12, loc='upper right')
    ax.legend([black_dot, black_filled_circle, black_filled_circle_lg], 
              [legend_1, legend_2, legend_3],frameon=False,borderaxespad=1,
              labelspacing=0.2, prop=legend_properties)


### Function to add the SIO logo

In [53]:
def add_sio_logo(fig, logo_file, xpos, ypos):

    # Convert the logo svg file to a png file with the
    # given scale and dpi
    logo_png = 'logo.png'
    svg2png(url=logo_file, write_to=logo_png, scale=10, dpi=300)

    logo = mpimg.imread(logo_png)

    fig.add_axes([xpos, ypos, 0.2, 0.2], anchor='SE', zorder=1)

    plt.imshow(logo)
    plt.axis('off')

    # Delete the logo png version
    os.remove(logo_png)


### Function to save the graphic

In [54]:
def save_graphic(fig, pdf_file, png_file):

    fig_width, fig_height = fig.get_size_inches()

    # For pdf
    width_in = 10
    height_in = (fig_height/fig_width) * width_in

    fig.set_size_inches(width_in, height_in)

    # Save to a high dpi so that logo png file has a high resolution
    fig.savefig(pdf_file, facecolor='w', edgecolor='w',
                orientation='landscape', format=None,
                transparent=False, bbox_inches='tight', dpi=600)

    # For png
    # Want a png of height 500px

    # First save figure as a png and then scale to size needed

    png_dpi = 300

    fig.savefig(png_file, facecolor='w', edgecolor='w',
                orientation='landscape', dpi=png_dpi, bbox_inches='tight')

    # Now resize the image to be width 1000px for the
    # keelingcurve website
    img = Image.open(png_file)
    img_width, img_height = img.size

    # width_px = 1000
    # img_scale = (width_px / img_width)
    # height_px = int((float(img_height) * float(img_scale)))

    height_px = 500
    img_scale = (height_px / img_height)
    width_px = int((float(img_width) * float(img_scale)))

    img = img.resize((width_px, height_px), Image.ANTIALIAS)
    img.save(png_file)


### Function to create a Context Manager

This allows a message to display in a cell while a function is running.
Usefull for indicating a function is running.

In [55]:
class ShowProgressContextManagerWidgets:

    def __init__(self, out, command=''):
        self.out = out
        self.command = command
        
    def __enter__(self):
        css_style = "<style>span.start { margin:0; padding: 0; color: blue; }</style>"
        html_str = f'<span class="start">Running {self.command}...</span>'

        self.out.clear_output()
        
        with self.out:
            display(HTML(css_style))
            display(HTML(html_str))

        
    def __exit__(self, exc_type, exc_value, exc_tb):
        
        css_style = "<style>span.end { margin: 0; padding: 0; color: green; }</style>"
        html_str = '<span class="end">Finished</span>'

        self.out.clear_output()

        with self.out:
            display(HTML(css_style)) 
            display(HTML(html_str))


# **Steps to create the graphic**

### Change any properties in the section [Functions to modify the graphic](#functions-to-modify-graphic)

### Comment out any function in the overall function [create_graphic](#function-to-create-the-graphic) to remove an element from the final graphic

Comment out a line by adding a # symbol in front of a statement


## **Set properties to use for matplotlib (plotting engine)**

In [56]:
set_matplotlib_properties()

## **Set plot limits and labeling**

### Set titles and axes labels

In [57]:
xlabel = ''
ylabel = "$\mathregular{CO}\\bf{_2}$" + " Concentration (ppm)"

title1 = ""
title2 = "Carbon dioxide concentration at Mauna Loa Observatory*"

### Set yaxis min and max limits

In [58]:
ymin = 405
ymax = 430

### Set xaxis min and max limits as both dates and decimals

In [59]:
now = datetime.now()
time_ago = now + relativedelta(months=-1)

xmin = time_ago
xmax = now

<a name="function-to-create-the-graphic"></a>
## **Function to create the graphic**

In [60]:
def create_graphic(msg_out, graphic_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, date_hourly, co2_hourly, date_daily, co2_daily, date_weekly, co2_weekly, btn):

    msg_out.clear_output()
    graphic_out.clear_output()

    # create a figure container
    fig = plt.figure()

    # Set axes
    # Add padding for exterior plot text
    ax = plt.axes([0.11, 0.12, 0.83, 0.54])

    # ---------------------------------------------------
    # Context manager 
    # 
    # Used to show messages while creating the animiation
    # ---------------------------------------------------

    with ShowProgressContextManagerWidgets(msg_out, 'Create Graphic') as progress:

        # ---------------------------------------------        
        # Convert decimal year to datetime for plotting
        # ---------------------------------------------  

        datetime_hourly = np.array([t2dt(x) for x in date_hourly])
        datetime_daily = np.array([t2dt(x) for x in date_daily])
        datetime_weekly = np.array([t2dt(x) for x in date_weekly])

        datetime_hourly_mko = np.array([t2dt(x) for x in date_hourly_mko])
        datetime_daily_mko = np.array([t2dt(x) for x in date_daily_mko])
        datetime_weekly_mko = np.array([t2dt(x) for x in date_weekly_mko])

        # -------------
        # Plot the data
        # -------------

        ax.plot(datetime_hourly, co2_hourly, 'o', color='black',markersize=4, 
                markerfacecolor='black', markeredgecolor='black', markeredgewidth=0, zorder=5)

        ax.plot(datetime_daily, co2_daily, 'o', color='black',markersize=10, 
                markerfacecolor='black', markeredgecolor='black', markeredgewidth=0, zorder=6)

        ax.plot(datetime_weekly, co2_weekly, 'o', color='black',markersize=12, 
                markerfacecolor='none', markeredgecolor='black', markeredgewidth=2.0, zorder=7)
        

        # differentiate MKO data

        blue2 = '#1b50d7'
        blue = '#1e47b0'

        ax.plot(datetime_hourly_mko, co2_hourly_mko, 'o', color=blue2,markersize=4, 
                markerfacecolor=blue2, markeredgecolor=blue2, markeredgewidth=0, zorder=10)

        ax.plot(datetime_daily_mko, co2_daily_mko, 'o', color=blue,markersize=10, 
                markerfacecolor=blue, markeredgecolor=blue, markeredgewidth=0, zorder=11)

        ax.plot(datetime_weekly_mko, co2_weekly_mko, 'o', color=blue,markersize=12, 
                markerfacecolor='none', markeredgecolor=blue, markeredgewidth=2.0, zorder=12)


        # -----------------------------------------------------
        # Change xaxis to use date labels and not decimal dates
        # -----------------------------------------------------
        create_xtick_labels(ax, xmin, xmax)

        # -------------------
        # Set plot properties
        # -------------------
        set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel)

        # ------------------------------------------------
        # Get todays date in various formats for labeling
        # ------------------------------------------------
        todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year = get_todays_date_variations()

        # ---------------
        # Add plot titles
        # ---------------
        add_plot_title(ax, title1, title2)

        # --------------
        # Add inset text
        # --------------
        add_inset_label(ax, 'Month', todays_date_modyyr)

        add_inset_label_mko(ax, "*Maunakea data is in blue")

        # ----------------
        # Add legend
        # ----------------
        add_legend_labels(ax)

        # --------------------
        # Add SIO logo to plot
        # --------------------
        xpos = 0.715
        ypos = 0.155

        add_sio_logo(fig, logo_file, xpos, ypos)

        # ------------
        # Save graphic
        # ------------
        fig = plt.gcf()

        save_graphic(fig, pdf_file, png_file)

    with graphic_out:

        plt.show()

<a name='button-to-create-and-save-the-graphic'></a>
## **Button to create and save the graphic**

In [61]:
msg_out = widgets.Output()
graphic_out = widgets.Output()

button = widgets.Button(description='Create Graphic', button_style='primary')
button.on_click(functools.partial(create_graphic, msg_out, graphic_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, date_hourly, co2_hourly, date_daily, co2_daily, date_weekly, co2_weekly))
display(button)

display(msg_out)
display(graphic_out)

Button(button_style='primary', description='Create Graphic', style=ButtonStyle())

Output()

Output()

## **Button to download PDF & PNG files of the graphic**

The download button only works when the notebook is run with Google Colab

If the notebook is run on a local machine, the graphics are inside the plots directory

In [62]:
def download_files(msg_out, btn):

    msg_out.clear_output()

    try:
        files.download(png_file)
        files.download(pdf_file)
    except:
        with msg_out:
            print("If running notebook in Google Colab:")
            print("Need to create the graphic first.")
            print(f"\nIf running in a python environment, the graphics are in the plots directory.")


msg_out = widgets.Output()

button = widgets.Button(description="Download Graphic Files", button_style="primary")

layout = widgets.Layout(width='auto', height='auto') #set width and height

button = widgets.Button(
    description='Download Graphic Files',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    button_style='primary',
    layout = layout
)   

button.on_click(functools.partial(download_files, msg_out))

display(button)
display(msg_out)

Button(button_style='primary', description='Download Graphic Files', layout=Layout(height='auto', width='auto'…

Output()